In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import explained_variance_score, r2_score
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from scipy.stats import pearsonr

# 1 Preparation

## 1.1 Embeddings

In [111]:
embeddings = pd.read_pickle('test/embeddings_complete.pickle')
len(embeddings)

10060

In [112]:
# combine 2 embeddings of father and mother for each individual
sample_id = []
gene_id = []
embedding = []
for i in range(0, len(embeddings), 2):
    sid = embeddings[i][0].split(':')[1]
    gid = embeddings[i][0].split(':')[2]
    sample_id.append(sid)
    gene_id.append(gid)
    combined_embedding = embeddings[i][1] + embeddings[i+1][1]    # sum embedding
    # combined_embedding = (embeddings[i][1] + embeddings[i+1][1])/2    # mean embedding
    # combined_embedding = np.concatenate([embeddings[i][1], embeddings[i+1][1]]) # concatenate 2 embeddings.
    embedding.append(combined_embedding)
    # order of mother and father should in theory not matter # TODO: does it make sense?
    # embedding.append(np.concatenate([embeddings[i+1][1], embeddings[i][1]]))
    # sample_id.append(sid)
    # gene_id.append(gid)
df = pd.DataFrame({"sample_id": sample_id, "gene_id": gene_id, "embedding": embedding})

In [113]:
df.head()

,sample_id,gene_id,embedding
0,NA20795,ENSG00000198502.5,"[0.020763932, -0.0043599964, 0.018072195, -0.0..."
1,HG00260,ENSG00000214425.1,"[0.034569502, 0.024124226, -0.06553179, -0.017..."
2,HG01632,ENSG00000176681.9,"[-0.0912664, 0.07260499, -0.07260455, 0.113591..."
3,HG00173,ENSG00000238083.3,"[0.08705197, 0.056551743, 0.033424325, -0.0829..."
4,HG00178,ENSG00000229450.2,"[0.14517619, -0.016366798, -0.0096497275, -0.0..."


## 1.2 Expression Data

In [6]:
expression = pd.read_csv("datasets/GD660.GeneQuantRPKM.txt.gz", sep="\t")
expression.head()

,TargetID,Gene_Symbol,Chr,Coord,HG00096.1.M_111124_6,HG00097.7.M_120219_2,HG00099.1.M_120209_6,HG00099.5.M_120131_3,HG00100.2.M_111215_8,HG00101.1.M_111124_4,...,NA20810.2.M_111215_7,NA20811.1.M_111124_5,NA20812.2.M_111216_6,NA20813.5.M_120131_1,NA20814.2.M_111215_6,NA20815.5.M_120131_5,NA20816.3.M_120202_7,NA20819.3.M_120202_2,NA20826.1.M_111124_1,NA20828.2.M_111216_8
0,ENSG00000225538.1,ENSG00000225538.1,11,55850277,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.0000,0.00000,0.00000,0.00000,0.0000,0.00000,0.00000,0.03876,0.00000
1,ENSG00000237851.1,ENSG00000237851.1,6,143109260,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.0000,0.00000,0.00000,0.00000,0.0000,0.00000,0.00000,0.00000,0.00000
2,ENSG00000243765.1,ENSG00000243765.1,15,58442766,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.0000,0.00000,0.00000,0.00000,0.0000,0.00000,0.00000,0.00000,0.00000
3,ENSG00000257527.1,ENSG00000257527.1,16,18505708,0.70561,0.66697,0.64004,0.26195,0.34695,1.49208,...,0.87085,0.9495,0.95837,0.51002,0.29422,0.2296,0.58671,0.27674,0.53630,0.17139
4,ENSG00000212855.5,ENSG00000212855.5,Y,9578193,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.0000,0.00000,0.00000,0.00000,0.0000,0.00000,0.00000,0.00000,0.00000


In [7]:
expression = expression[expression["Gene_Symbol"].isin(df["gene_id"].values)]
expression = pd.melt(expression, id_vars=["TargetID", "Gene_Symbol", "Chr", "Coord"], var_name="sample_id", value_name="expression")
expression["sample_id"] = expression["sample_id"].str.split(".", expand=True)[0]
expression.head()

,TargetID,Gene_Symbol,Chr,Coord,sample_id,expression
0,ENSG00000232629.4,ENSG00000232629.4,6,32731311,HG00096,9.96245
1,ENSG00000179344.11,ENSG00000179344.11,6,32636160,HG00096,43.78876
2,ENSG00000176681.9,ENSG00000176681.9,17,44370099,HG00096,6.34823
3,ENSG00000214425.1,ENSG00000214425.1,17,43595264,HG00096,0.40868
4,ENSG00000237541.3,ENSG00000237541.3,6,32709119,HG00096,51.03589


Prepare data for Expression prediction.

y = expression values for (sample, gene) pairs.

X = matrix containing the embeddings for each (sample, gene) pair.

In [114]:
df = pd.merge(df, expression, left_on=["sample_id", "gene_id"], right_on=["sample_id", "Gene_Symbol"])
print(f"Number of samples for prediction expression: {len(df)}")

Number of samples for prediction expression: 4830


In [115]:
genes = df.gene_id.unique()
print(f"Number of genes: {len(genes)}")

Number of genes: 10


In [153]:
gene = genes[9]
y = df[df.gene_id==gene].expression.values
X = np.stack(df[df.gene_id==gene].embedding.values)

In [129]:
# y = df["expression"].values
# X = np.stack(df["embedding"].values)

In [156]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# 2 Prediction of Expression

## 2.1 Linear Regression

In [157]:
linear_regression_model = LinearRegression().fit(X_train, y_train)

In [158]:
predicted = linear_regression_model.predict(X_test)
print(f"R² of test set: {r2_score(y_test, predicted)}")
print(f"Explained variance score on test set: {explained_variance_score(y_test, predicted)}")
print(f"R value (Pearson correlation coefficient): {pearsonr(y_test, predicted)[0]}")

R² of test set: -1.5831126804131839
Explained variance score on test set: -1.3257274957999572
R value (Pearson correlation coefficient): 0.32652405054016787


## 2.2 Ridge Regression

In [55]:
ridge_regression_model = Ridge(alpha=1.0).fit(X_train, y_train)

In [56]:
predicted = ridge_regression_model.predict(X_test)
print(f"R² of test set: {r2_score(y_test, predicted)}")
print(f"Explained variance score on test set: {explained_variance_score(y_test, predicted)}")
print(f"R value (Pearson correlation coefficient): {pearsonr(y_test, predicted)[0]}")

R² of test set: 0.20036904113714638
Explained variance score on test set: 0.21356514640780466
R value (Pearson correlation coefficient): 0.4712295321529268


## 2.3 Lasso Regression

In [57]:
lasso_regression_model = Lasso(alpha=1.0).fit(X_train, y_train)

In [58]:
predicted = lasso_regression_model.predict(X_test)
print(f"R² of test set: {r2_score(y_test, predicted)}")
print(f"Explained variance score on test set: {explained_variance_score(y_test, predicted)}")
print(f"R value (Pearson correlation coefficient): {pearsonr(y_test, predicted)[0]}")

R² of test set: -0.015638000638453553
Explained variance score on test set: 2.220446049250313e-16
R value (Pearson correlation coefficient): nan


/home/stockhaus/miniconda3/envs/sysgen/lib/python3.10/site-packages/scipy/stats/_stats_py.py:4781: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))


## 2.4 Elastic Net Regression

In [61]:
elasticnet_regression_model = ElasticNet(random_state=53).fit(X_train, y_train)

In [62]:
predicted = elasticnet_regression_model.predict(X_test)
print(f"R² of test set: {r2_score(y_test, predicted)}")
print(f"Explained variance score on test set: {explained_variance_score(y_test, predicted)}")
print(f"R value (Pearson correlation coefficient): {pearsonr(y_test, predicted)[0]}")

R² of test set: -0.015638000638453553
Explained variance score on test set: 2.220446049250313e-16
R value (Pearson correlation coefficient): nan


/home/stockhaus/miniconda3/envs/sysgen/lib/python3.10/site-packages/scipy/stats/_stats_py.py:4781: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))


## 2.5 Support Vector Machine

In [63]:
svr = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2)).fit(X_train, y_train)

In [64]:
predicted = svr.predict(X_test)
print(f"R² of test set: {r2_score(y_test, predicted)}")
print(f"Explained variance score on test set: {explained_variance_score(y_test, predicted)}")
print(f"R value (Pearson correlation coefficient): {pearsonr(y_test, predicted)[0]}")

R² of test set: 0.09475847920011782
Explained variance score on test set: 0.21356630610002958
R value (Pearson correlation coefficient): 0.49284726089483377


# Comparison to prediction based on single SNPs

In [17]:
reference = pd.read_csv("datasets/EUR373.gene.cis.FDR5.all.rs137.txt.gz", sep="\t")
reference

,SNP_ID,ID,GENE_ID,PROBE_ID,CHR_SNP,CHR_GENE,SNPpos,TSSpos,distance,rvalue,pvalue,log10pvalue
0,rs1566523,-,ENSG00000142794.11,ENSG00000142794.11,1,1,21766663.0,21766621,42.0,-0.782786,3.883913e-78,77.410730
1,rs66732917,-,ENSG00000142794.11,ENSG00000142794.11,1,1,21766608.0,21766621,13.0,-0.775971,5.736684e-76,75.241339
2,rs12734589,-,ENSG00000142794.11,ENSG00000142794.11,1,1,21760127.0,21766621,6494.0,-0.753190,3.094123e-69,68.509462
3,rs4654745,-,ENSG00000142794.11,ENSG00000142794.11,1,1,21767096.0,21766621,475.0,-0.753071,3.339200e-69,68.476358
4,rs10916990,-,ENSG00000142794.11,ENSG00000142794.11,1,1,21763132.0,21766621,3489.0,-0.750913,1.327049e-68,67.877113
...,...,...,...,...,...,...,...,...,...,...,...,...
419978,rs133379,-,ENSG00000198951.6,ENSG00000198951.6,22,22,42468535.0,42466846,1689.0,0.231022,6.568113e-06,5.182559
419979,rs4253766,-,ENSG00000075234.12,ENSG00000075234.12,22,22,46623905.0,46663858,39953.0,-0.231008,6.576526e-06,5.182003
419980,rs73165152,-,ENSG00000100417.7,ENSG00000100417.7,22,22,42340397.0,41985894,354503.0,-0.231006,6.577641e-06,5.181930
419981,rs133352,-,ENSG00000198951.6,ENSG00000198951.6,22,22,42434022.0,42466846,32824.0,0.231003,6.579434e-06,5.181811


In [21]:
print(reference[reference.GENE_ID==gene].rvalue.max())
print(reference[reference.GENE_ID==gene].rvalue.min())

0.287653816193465
-0.584827960561856
